In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [2]:
file_path = "..\csv_completos_limpio\compras_completo_limpio.csv"
df = pd.read_csv(file_path)
df

,identificador,nombre,area,provincia,comunidad_autonoma,codigo_postal,coordenadas,href,antiguedad,conservacion,...,precio,precio_m2,emisiones,consumo,orientacion,calle_alumbrada,agencia,actualizacion,timestamp,tipo_de_casa
0,51748300470_101800,"Casa adosada en calle de la Gran Vía, 7",Corredor Del Henares,Madrid,Comunidad De Madrid,NaN,"40.5438, -3.37927",https://www.pisos.com/comprar/casa_adosada-cam...,NaN,NaN,...,323300.0,2694.0,"A, A","A, A",Sin especificar,Sin especificar,Hercesa,Anuncio actualizado el 28/02/2025,2025-02-28 21:02:41,Casa adosada
1,38350600856_109700,"Casa adosada en calle de la Gran Vía, 12",Corredor Del Henares,Madrid,Comunidad De Madrid,NaN,"40.54455876322138, -3.3799266815185547",https://www.pisos.com/comprar/casa_adosada-cam...,NaN,A estrenar,...,340200.0,2858.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,Hercesa,Anuncio actualizado el 08/02/2025,2025-02-28 21:02:42,Casa adosada
2,49206134954_100200,Casa adosada en Villalbilla,Corredor Del Henares,Madrid,Comunidad De Madrid,NaN,"40.4464284, -3.3478297",https://www.pisos.com/comprar/casa_adosada-vil...,Entre 20 y 30 años,A reformar,...,322000.0,1504.0,"E, E","E, E",Sin especificar,Sin especificar,Estudio Villalbilla-El Viso Sl,Anuncio actualizado el 29/01/2025,2025-02-28 21:02:42,Casa adosada
3,35856717666_109700,"Casa adosada en calle de la Gran Vía, 12",Corredor Del Henares,Madrid,Comunidad De Madrid,NaN,"40.54455876322138, -3.3799266815185547",https://www.pisos.com/comprar/casa_adosada-cam...,NaN,A estrenar,...,338370.0,2843.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,Hercesa,Anuncio actualizado el 08/02/2025,2025-02-28 21:02:42,Casa adosada
4,51761494757_995907,Chalet unifamiliar en calle de Soto del Henares,Corredor Del Henares,Madrid,Comunidad De Madrid,NaN,"40.4304711, -3.2924094",https://www.pisos.com/comprar/chalet_unifamili...,NaN,En buen estado,...,320000.0,1777.0,"A, A","A, A",Sin especificar,Sin especificar,AVANCE URBANO,Anuncio actualizado el 13/02/2025,2025-02-28 21:02:42,Chalet unifamiliar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399809,48431394446_100500,Piso en calle -,Melilla,Melilla,Melilla,52005.0,"35.2826632, -2.9449037",https://www.pisos.com/comprar/piso-melilla_cen...,NaN,En buen estado,...,105000.0,1693.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,Urbana Consulting Grupo XXI,Anuncio actualizado el 16/01/2025,2025-03-03 15:13:24,Piso
399810,48370841753_108700,Piso en Melilla,Melilla,Melilla,Melilla,52006.0,"35.278731151, -2.937359951",https://www.pisos.com/comprar/piso-melilla_cen...,Entre 30 y 50 años,NaN,...,195000.0,2532.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,IAD ESPAÑA,Anuncio actualizado el 07/12/2024,2025-03-03 15:13:24,Piso
399811,50838094110_100500,Piso en calle del Gran Capitán,Melilla,Melilla,Melilla,52002.0,"35.2959994, -2.9441408",https://www.pisos.com/comprar/piso-melilla_cen...,NaN,En buen estado,...,158000.0,2164.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,Urbana Consulting Grupo XXI,Anuncio actualizado el 29/01/2025,2025-03-03 15:13:24,Piso
399812,48395661430_100500,Piso en calle de Castelar,Melilla,Melilla,Melilla,52001.0,"35.2958781, -2.9415108",https://www.pisos.com/comprar/piso-melilla_cen...,NaN,En buen estado,...,116000.0,1288.0,Error al procesar emisiones,Error al procesar consumo,Sin especificar,Sin especificar,Urbana Consulting Grupo XXI,Anuncio actualizado el 14/11/2024,2025-03-03 15:13:24,Piso


In [6]:
nan_counts = df.isna().sum()
nan_counts
df["antiguedad"].unique()

array([nan, 'Entre 20 y 30 años', 'Entre 5 y 10 años', 'Menos de 5 años',
       'Entre 30 y 50 años', 'Más de 50 años', 'Entre 10 y 20 años'],
      dtype=object)

In [3]:
df["latitud"] = df["coordenadas"].str.split(",").str[0].str.strip().astype(float, errors='ignore')
df["longitud"] = df["coordenadas"].str.split(",").str[1].str.strip().astype(float, errors='ignore')

In [ ]:
columnas_clustering = ["precio", "antiguedad", "conservacion", "latitud", "longitud", "piscina", "baños","habitaciones","superficie","superficie_util","aire_acondicionado"]

In [5]:
df2 = df[columnas_clustering].copy()
df2

,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
0,323300.0,2694.0,NaN,NaN,40.543800,-3.379270,Si,3.0,3.0,120.0,120.0,Sin especificar
1,340200.0,2858.0,NaN,A estrenar,40.544559,-3.379927,Sin especificar,2.0,3.0,119.0,119.0,Sin especificar
2,322000.0,1504.0,Entre 20 y 30 años,A reformar,40.446428,-3.347830,Si,2.0,4.0,214.0,214.0,Si
3,338370.0,2843.0,NaN,A estrenar,40.544559,-3.379927,Sin especificar,2.0,3.0,119.0,119.0,Sin especificar
4,320000.0,1777.0,NaN,En buen estado,40.430471,-3.292409,Sin especificar,3.0,4.0,180.0,152.0,Si
...,...,...,...,...,...,...,...,...,...,...,...,...
399809,105000.0,1693.0,NaN,En buen estado,35.282663,-2.944904,Sin especificar,1.0,2.0,62.0,62.0,Sin especificar
399810,195000.0,2532.0,Entre 30 y 50 años,NaN,35.278731,-2.937360,Sin especificar,2.0,3.0,77.0,77.0,Sin especificar
399811,158000.0,2164.0,NaN,En buen estado,35.295999,-2.944141,Sin especificar,2.0,2.0,73.0,73.0,Sin especificar
399812,116000.0,1288.0,NaN,En buen estado,35.295878,-2.941511,Sin especificar,1.0,2.0,90.0,90.0,Sin especificar


In [6]:
nan_counts = df2.isna().sum()
nan_counts
#df["antiguedad"].unique()

precio                     0
precio_m2               1119
antiguedad            182613
conservacion          226836
latitud                    0
longitud                   0
piscina                    0
baños                  17732
habitaciones           17042
superficie              4836
superficie_util         4836
aire_acondicionado         0
dtype: int64

In [6]:
# Eliminar filas con valores NaN en cualquier columna
df2 = df2.dropna()
df2


,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
2,322000.0,1504.0,Entre 20 y 30 años,A reformar,40.446428,-3.347830,Si,2.0,4.0,214.0,214.0,Si
10,259900.0,2384.0,Entre 30 y 50 años,A reformar,40.424559,-3.556857,Sin especificar,2.0,4.0,109.0,109.0,Sin especificar
16,299900.0,2054.0,Más de 50 años,En buen estado,40.455748,-3.476452,Sin especificar,1.0,3.0,146.0,146.0,Sin especificar
21,795000.0,1717.0,Entre 30 y 50 años,En buen estado,40.421404,-3.555159,Sin especificar,3.0,5.0,463.0,463.0,Si
25,386000.0,2859.0,Entre 10 y 20 años,En buen estado,40.502241,-3.516537,Sin especificar,2.0,3.0,135.0,135.0,Sin especificar
...,...,...,...,...,...,...,...,...,...,...,...,...
399782,350000.0,2258.0,Entre 10 y 20 años,En buen estado,35.290371,-2.942934,Sin especificar,3.0,3.0,155.0,155.0,Si
399783,450000.0,2960.0,Menos de 5 años,En buen estado,35.290508,-2.939871,Si,3.0,4.0,152.0,152.0,Si
399789,120000.0,1333.0,Entre 20 y 30 años,En buen estado,35.299989,-2.948280,Sin especificar,2.0,2.0,90.0,80.0,Si
399800,190000.0,2134.0,Entre 10 y 20 años,En buen estado,35.274124,-2.945657,Sin especificar,2.0,3.0,89.0,80.0,Sin especificar


In [8]:
#no usar demomento
encoder = OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(df2[['piscina']])
df2_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['piscina']))
df2 = pd.concat([df2.drop(columns=['piscina']), df2_encoded], axis=1)


In [10]:
df2

,precio,precio_m2,antiguedad,conservacion,latitud,longitud,baños,habitaciones,superficie,superficie_util,aire_acondicionado,piscina_No,piscina_Si,piscina_Sin especificar
2,322000.0,1504.0,Entre 20 y 30 años,A reformar,40.446428,-3.347830,2.0,4.0,214.0,214.0,Si,0.0,0.0,1.0
10,259900.0,2384.0,Entre 30 y 50 años,A reformar,40.424559,-3.556857,2.0,4.0,109.0,109.0,Sin especificar,0.0,1.0,0.0
16,299900.0,2054.0,Más de 50 años,En buen estado,40.455748,-3.476452,1.0,3.0,146.0,146.0,Sin especificar,0.0,0.0,1.0
21,795000.0,1717.0,Entre 30 y 50 años,En buen estado,40.421404,-3.555159,3.0,5.0,463.0,463.0,Si,0.0,0.0,1.0
25,386000.0,2859.0,Entre 10 y 20 años,En buen estado,40.502241,-3.516537,2.0,3.0,135.0,135.0,Sin especificar,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
94321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0
94322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
94323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0


In [7]:
label_encoder = LabelEncoder()
df2['antiguedad'] = label_encoder.fit_transform(df2['antiguedad'])
df2

C:\Users\kikeg\AppData\Local\Temp\ipykernel_14328\1146099926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['antiguedad'] = label_encoder.fit_transform(df2['antiguedad'])


,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
2,322000.0,1504.0,1,A reformar,40.446428,-3.347830,Si,2.0,4.0,214.0,214.0,Si
10,259900.0,2384.0,2,A reformar,40.424559,-3.556857,Sin especificar,2.0,4.0,109.0,109.0,Sin especificar
16,299900.0,2054.0,5,En buen estado,40.455748,-3.476452,Sin especificar,1.0,3.0,146.0,146.0,Sin especificar
21,795000.0,1717.0,2,En buen estado,40.421404,-3.555159,Sin especificar,3.0,5.0,463.0,463.0,Si
25,386000.0,2859.0,0,En buen estado,40.502241,-3.516537,Sin especificar,2.0,3.0,135.0,135.0,Sin especificar
...,...,...,...,...,...,...,...,...,...,...,...,...
399782,350000.0,2258.0,0,En buen estado,35.290371,-2.942934,Sin especificar,3.0,3.0,155.0,155.0,Si
399783,450000.0,2960.0,4,En buen estado,35.290508,-2.939871,Si,3.0,4.0,152.0,152.0,Si
399789,120000.0,1333.0,1,En buen estado,35.299989,-2.948280,Sin especificar,2.0,2.0,90.0,80.0,Si
399800,190000.0,2134.0,0,En buen estado,35.274124,-2.945657,Sin especificar,2.0,3.0,89.0,80.0,Sin especificar


In [8]:
nan_counts = df2.isna().sum()
nan_counts
df["antiguedad"].unique()

array([nan, 'Entre 20 y 30 años', 'Entre 5 y 10 años', 'Menos de 5 años',
       'Entre 30 y 50 años', 'Más de 50 años', 'Entre 10 y 20 años'],
      dtype=object)

In [9]:
label_encoder = LabelEncoder()
df2['conservacion'] = label_encoder.fit_transform(df2['conservacion'])
df2

C:\Users\kikeg\AppData\Local\Temp\ipykernel_14328\3095357990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['conservacion'] = label_encoder.fit_transform(df2['conservacion'])


,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
2,322000.0,1504.0,1,1,40.446428,-3.347830,Si,2.0,4.0,214.0,214.0,Si
10,259900.0,2384.0,2,1,40.424559,-3.556857,Sin especificar,2.0,4.0,109.0,109.0,Sin especificar
16,299900.0,2054.0,5,2,40.455748,-3.476452,Sin especificar,1.0,3.0,146.0,146.0,Sin especificar
21,795000.0,1717.0,2,2,40.421404,-3.555159,Sin especificar,3.0,5.0,463.0,463.0,Si
25,386000.0,2859.0,0,2,40.502241,-3.516537,Sin especificar,2.0,3.0,135.0,135.0,Sin especificar
...,...,...,...,...,...,...,...,...,...,...,...,...
399782,350000.0,2258.0,0,2,35.290371,-2.942934,Sin especificar,3.0,3.0,155.0,155.0,Si
399783,450000.0,2960.0,4,2,35.290508,-2.939871,Si,3.0,4.0,152.0,152.0,Si
399789,120000.0,1333.0,1,2,35.299989,-2.948280,Sin especificar,2.0,2.0,90.0,80.0,Si
399800,190000.0,2134.0,0,2,35.274124,-2.945657,Sin especificar,2.0,3.0,89.0,80.0,Sin especificar


In [17]:
nan_counts = df2.isna().sum()
nan_counts

precio                     68027
precio_m2                  68027
antiguedad                     0
conservacion                   0
latitud                    68027
longitud                   68027
baños                      68027
habitaciones               68027
superficie                 68027
superficie_util            68027
aire_acondicionado         68027
piscina_No                 68027
piscina_Si                 68027
piscina_Sin especificar    68027
dtype: int64

In [10]:
label_encoder = LabelEncoder()
df2['aire_acondicionado'] = label_encoder.fit_transform(df2['aire_acondicionado'])
df2

C:\Users\kikeg\AppData\Local\Temp\ipykernel_14328\1701326482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['aire_acondicionado'] = label_encoder.fit_transform(df2['aire_acondicionado'])


,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
2,322000.0,1504.0,1,1,40.446428,-3.347830,Si,2.0,4.0,214.0,214.0,1
10,259900.0,2384.0,2,1,40.424559,-3.556857,Sin especificar,2.0,4.0,109.0,109.0,2
16,299900.0,2054.0,5,2,40.455748,-3.476452,Sin especificar,1.0,3.0,146.0,146.0,2
21,795000.0,1717.0,2,2,40.421404,-3.555159,Sin especificar,3.0,5.0,463.0,463.0,1
25,386000.0,2859.0,0,2,40.502241,-3.516537,Sin especificar,2.0,3.0,135.0,135.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
399782,350000.0,2258.0,0,2,35.290371,-2.942934,Sin especificar,3.0,3.0,155.0,155.0,1
399783,450000.0,2960.0,4,2,35.290508,-2.939871,Si,3.0,4.0,152.0,152.0,1
399789,120000.0,1333.0,1,2,35.299989,-2.948280,Sin especificar,2.0,2.0,90.0,80.0,1
399800,190000.0,2134.0,0,2,35.274124,-2.945657,Sin especificar,2.0,3.0,89.0,80.0,2


In [11]:
label_encoder = LabelEncoder()
df2['piscina'] = label_encoder.fit_transform(df2['piscina'])
df2

C:\Users\kikeg\AppData\Local\Temp\ipykernel_14328\457218677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['piscina'] = label_encoder.fit_transform(df2['piscina'])


,precio,precio_m2,antiguedad,conservacion,latitud,longitud,piscina,baños,habitaciones,superficie,superficie_util,aire_acondicionado
2,322000.0,1504.0,1,1,40.446428,-3.347830,1,2.0,4.0,214.0,214.0,1
10,259900.0,2384.0,2,1,40.424559,-3.556857,2,2.0,4.0,109.0,109.0,2
16,299900.0,2054.0,5,2,40.455748,-3.476452,2,1.0,3.0,146.0,146.0,2
21,795000.0,1717.0,2,2,40.421404,-3.555159,2,3.0,5.0,463.0,463.0,1
25,386000.0,2859.0,0,2,40.502241,-3.516537,2,2.0,3.0,135.0,135.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
399782,350000.0,2258.0,0,2,35.290371,-2.942934,2,3.0,3.0,155.0,155.0,1
399783,450000.0,2960.0,4,2,35.290508,-2.939871,1,3.0,4.0,152.0,152.0,1
399789,120000.0,1333.0,1,2,35.299989,-2.948280,2,2.0,2.0,90.0,80.0,1
399800,190000.0,2134.0,0,2,35.274124,-2.945657,2,2.0,3.0,89.0,80.0,2


In [12]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df2['Cluster'] = kmeans.fit_predict(df2)

C:\Users\kikeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\kikeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^

In [17]:
df2["Cluster"].value_counts()

Cluster
0    72032
2    18963
3     2885
1      445
Name: count, dtype: int64

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df2[["precio", "precio_m2", "antiguedad", "conservacion", "latitud", "longitud", "piscina", "baños","habitaciones","superficie","superficie_util","aire_acondicionado"]])

In [18]:
X = df2.drop(columns=["precio"])  # Variables predictoras (incluye "Cluster")
y = df2["precio"]  # Variable objetivo (precio de la vivienda)

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Crear y entrenar el modelo
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Evaluar el modelo
score = modelo.score(X_test, y_test)
print(f"Precisión del modelo: {score:.2f}")
train_score = modelo.score(X_train, y_train)
test_score = modelo.score(X_test, y_test)
print(f"Precisión en entrenamiento: {train_score:.2f}")
print(f"Precisión en test: {test_score:.2f}")
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Raíz del MSE

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")

Precisión del modelo: 0.99
Precisión en entrenamiento: 1.00
Precisión en test: 0.99
MAE: 3483.92
MSE: 1637135997.78
RMSE: 40461.54


In [22]:
#este ejmeplo vale 115k
import numpy as np

nuevo_dato = np.array([[798, 2, 1, 40.4168 ,-3.7038, 1, 2, 4, 140, 120, 1]])  # Datos nuevos # Aplicar la misma transformación
prediccion_precio = modelo.predict(nuevo_dato)
print(f"Precio estimado: {prediccion_precio[0]:,.2f} €")

C:\Users\kikeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


ValueError: X has 11 features, but RandomForestRegressor is expecting 12 features as input.

In [ ]:
modelo2 = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo2.fit(X_train_scaled, y_train)
score = modelo.score(X_test_scaled, y_test)
print(f"Precisión del modelo con escalado: {score:.2f}")

Precisión del modelo con escalado: 0.01


C:\Users\kikeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo.fit(X_train_scaled, y_train)
score = modelo.score(X_test_scaled, y_test)
print(f"Precisión del modelo con escalado: {score:.2f}")

Precisión del modelo con escalado: 0.98


In [ ]:
#este ejmeplo vale 115k
import numpy as np
nuevo_dato = np.array([[798, 2, 1, 40.4168 ,-3.7038, 1, 2, 4, 140, 120, 1]])  # Datos nuevos
nuevo_dato_escalado = scaler.transform(nuevo_dato)  # Aplicar la misma transformación
prediccion_precio = modelo.predict(nuevo_dato_escalado)
print(f"Precio estimado: {prediccion_precio[0]:,.2f} €")

Precio estimado: 124,385.00 €


C:\Users\kikeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


: 

In [ ]:
df2

In [ ]:
columnas_clustering = ["precio", "precio_m2", "antiguedad", "conservacion", "latitud", "longitud", "piscina", "baños","habitaciones","superficie","superficie_util","aire_acondicionado"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

model3 = LinearRegression()
model3.fit(X_train, y_train)

# Hacer predicciones
y_pred = model3.predict(X_test)

# Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results = pd.DataFrame({'Métrica': ['MAE', 'R²'], 'Valor': [mae, r2]})


In [ ]:
results

,Métrica,Valor
0,MAE,178268.503605
1,R²,0.392489


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

X_scaled = scaler.fit_transform(X)

    # Dividir en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Inicializar y entrenar el modelo de Regresión Lineal
model4 = LinearRegression()
model4.fit(X_train, y_train)

    # Hacer predicciones
y_pred = model4.predict(X_test)

    # Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

    # Mostrar resultados
results_scaled2 = pd.DataFrame({'Métrica': ['MAE', 'R²'], 'Valor': [mae, r2]})

In [ ]:
results_scaled2

,Métrica,Valor
0,MAE,178268.503605
1,R²,0.392489


In [ ]:

from xgboost import XGBRegressor


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Inicializar y entrenar el modelo XGBoost
model5 = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model5.fit(X_train, y_train)

    # Hacer predicciones
y_pred = model5.predict(X_test)

    # Evaluar el modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
results_xgb = pd.DataFrame({'Métrica': ['MAE', 'R²'], 'Valor': [mae, r2]})



In [ ]:
results_xgb


,Métrica,Valor
0,MAE,11099.251122
1,R²,0.978904
